In [ ]:
import pandas as pd
import numpy as np
import torch
import gc
import matplotlib.pyplot as plt
import extrametrics as em
from neuralforecast.core import NeuralForecast
from neuralforecast.losses.pytorch import MSE, MAE, MAPE, RMSE
from neuralforecast.losses.numpy import mse, mae, mape, rmse
from neuralforecast.models import StemGNN
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import os
from tqdm import tqdm

In [ ]:
Y_df = pd.read_csv('m5_stemgnn_smallest_weekly.csv')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
Y_df.head()

In [ ]:
minmax_scaler = MinMaxScaler()

# Step 2: Group by unique_id to normalize each product's timeseries individually
def apply_minmax(group):
    # MinMax scaling
    group['y_minmax'] = minmax_scaler.fit_transform(group[['y']])
    return group


# Apply MinMax normalization
Y_df_minmax = Y_df.groupby('unique_id', group_keys=False).apply(apply_minmax)


# Drop the original 'y' column and rename the normalized columns
Y_df_minmax = Y_df_minmax.drop(columns=['y']).rename(columns={'y_minmax': 'y'})

print(Y_df_minmax.head())


In [ ]:
# We make validation and test splits
n_time = len(Y_df.ds.unique())
val_size = int(.2 * n_time)
test_size = int(.1 * n_time)
timeseries_count = len(Y_df.unique_id.unique())

n_time, val_size, test_size, timeseries_count

In [ ]:
model = StemGNN(
        h=13,
        input_size=13,
        n_series=timeseries_count,
        scaler_type='robust',
        max_steps=100,
        early_stop_patience_steps=-1,
        val_check_steps=1,
        learning_rate=1e-2,
        loss=MAE(),
        valid_loss=None,
        batch_size=8,
        random_seed = 1
    )

In [ ]:
def metrics_eval_grid(model, Y_hat_df):
    mae_model = mae(Y_hat_df['y'], Y_hat_df[f'{model}'])
    mse_model = mse(Y_hat_df['y'], Y_hat_df[f'{model}'])
    mape_model = mape(Y_hat_df['y'], Y_hat_df[f'{model}'])
    rmse_model = rmse(Y_hat_df['y'], Y_hat_df[f'{model}'])
    wmape_model = em.wmape(Y_hat_df['y'], Y_hat_df[f'{model}'])
    r_squared_model = em.r_squared(Y_hat_df['y'], Y_hat_df[f'{model}'])

    return mae_model, mse_model, rmse_model, mape_model, wmape_model, r_squared_model


In [ ]:
# Step 1: Create and fit the model
nf = NeuralForecast(models=[model], freq='W')
Y_hat_df = nf.cross_validation(df=Y_df_minmax, val_size=val_size, test_size=test_size, n_windows=None)                                 
Y_hat_df = Y_hat_df.reset_index()
Y_hat_df


In [ ]:
y_mae, y_mse, y_rmse, y_mape, y_wmape, y_r_squared = metrics_eval_grid(model, Y_hat_df)
print(f"MAE: {y_mae:.3f}, MSE: {y_mse:.3f}, RMSE: {y_rmse:.3f}, "
                  f"MAPE: {y_mape:.3f}%, WMAPE: {y_wmape:.3f}%, R_Squared: {y_r_squared:.3f}%\n")


In [ ]:
path = './checkpoints/' 
nf.save(path=path, model_index=None, overwrite=True, save_dataset=True)

In [ ]:
# Initialize output dataframe with weekly frequency
output_df = Y_df.copy()
HORIZON = 13  # 13 weeks
n_series = Y_df['unique_id'].nunique()

# Find common window range across all series
group_sizes = Y_df_minmax.groupby('unique_id').size()
max_window_start = group_sizes.min() - 2 * HORIZON
window_starts = range(max_window_start + 1)
n_windows = len(window_starts)

# Create weekly aligned windows using sliding_window_view
window_data = []
for uid, group in Y_df_minmax.groupby('unique_id'):
    weekly_dates = group['ds'].values
    y_values = group['y'].values
    
    # Create weekly windows
    windows = np.lib.stride_tricks.sliding_window_view(y_values, 2 * HORIZON)[:n_windows]
    
    window_data.append({
        'uid': uid,
        'input_windows': windows[:, :HORIZON],
        'pred_windows': windows[:, HORIZON:],
        'input_dates': [weekly_dates[i:i+HORIZON] for i in window_starts],
        'pred_dates': [weekly_dates[i+HORIZON:i+2*HORIZON] for i in window_starts]
    })

# Create batched prediction dataframe with weekly alignment
predict_records = []
for w in window_data:
    for i in range(n_windows):
        for week_idx in range(HORIZON):
            predict_records.append({
                'unique_id': f"{w['uid']}_window_{i}",
                'ds': w['input_dates'][i][week_idx],  # Maintain weekly dates
                'y': w['input_windows'][i, week_idx]
            })

window_df = pd.DataFrame(predict_records)
Y_hat = nf.predict(window_df)
pred_lookup = Y_hat.groupby('unique_id')['StemGNN'].apply(list).to_dict()

# Process predictions with weekly alignment
scaler_dict = Y_df.groupby('unique_id')['y'].agg(['min', 'max']).to_dict('index')
output_df = output_df.set_index(['unique_id', 'ds'])

for w in tqdm(window_data, desc='Weekly Windows'):
    uid = w['uid']
    scaler = scaler_dict[uid]
    
    for i in range(n_windows):
        col_X = f'X_{i}'
        col_Y = f'Y_{i}'
        
        # Input values
        input_dates = w['input_dates'][i]
        output_df.loc[(uid, input_dates), col_X] = Y_df.loc[
            (Y_df['unique_id'] == uid) & (Y_df['ds'].isin(input_dates)), 'y'
        ].values
        
        # Predictions
        pred_key = f"{uid}_window_{i}"
        preds = np.array(pred_lookup.get(pred_key, [np.nan]*HORIZON))
        denorm_preds = preds * (scaler['max'] - scaler['min']) + scaler['min']
        denorm_preds = np.round(np.clip(denorm_preds, 0, None)).astype(int)
        
        pred_dates = w['pred_dates'][i]
        output_df.loc[(uid, pred_dates), col_Y] = denorm_preds[:len(pred_dates)]

# Final formatting
output_df = output_df.reset_index()
output_df['ds'] = output_df['ds'].dt.strftime('%Y-%m-%d')
output_df = output_df[[c for c in output_df.columns if not c.startswith('level')]]

output_df.to_csv('weekly_output.csv', index=False)
print(output_df.head())